# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [3]:
# Settings
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Utility imports
import os
import copy
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook
from PIL import Image
from collections import OrderedDict, namedtuple

# DL imports
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [4]:
# Setting the path to the dataset
data_dir = 'flowers'

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [5]:
def load_data_from_dir(data_dir_path):
    """
    Loads and preprocesses the data. Expects path to a directory
    with train, valid, and test subfolders containing class examples.
    
    Return data dictionary with the preprocessed data.
    """
    
    # Creating default data transformations
    data_transforms = {
        'train':transforms.Compose([transforms.RandomRotation(30),
                                    transforms.RandomResizedCrop(224),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.RandomVerticalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], 
                                                         [0.229, 0.224, 0.225])]),
        'valid':transforms.Compose([
                                    transforms.Resize(255),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], 
                                                         [0.229, 0.224, 0.225])]),
        'test':transforms.Compose([
                                    transforms.Resize(255),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], 
                                                         [0.229, 0.224, 0.225])])
    }


    # Getting the images
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                              data_transforms[x])
                      for x in ['train', 'valid', 'test']}
    
    # Creating data loaders to serve batches to the model
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64,
                                                 shuffle=True)
                  for x in ['train', 'valid', 'test']}
    
    # Saving helpful information about the datasets e.g. sizes, class names
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid', 'test']}
    class_names = image_datasets['train'].classes
    
    # creating a dictionary to return
    return {
        'data_dir_path':data_dir_path,
        'image_datasets':image_datasets, 
        'dataloaders':dataloaders, 
        'dataset_sizes':dataset_sizes, 
        'class_names':class_names
    }

In [6]:
# Loading the data
data = load_data_from_dir(data_dir)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [7]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# inspecting the dictionary
cat_to_name['1']

'pink primrose'

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [8]:
def load_pretrained_model(arch='resnet18'):
    """
    Loads user specified architecture. 
    Currently supported:
    * resnet18
    * resnet152
    """
    
    if arch=='resnet18':
        model = models.resnet18(pretrained=True) 
    elif arch=='resnet152':
        model = models.resnet152(pretrained=True)                               
    else:
        raise Exception('Not able to recognize requested model architecture')
            
    return model


def replace_classifier(model, n_classes=102, hidden_units=300):
    """
    Replaces the pretrained model's classifier with a custom
    classifier to fit the new dataset
    """
    
    # Freezing the pre-trained parameters
    for p in model.parameters():
        p.requires_grad = False

    # Building a new classifier
    clf = nn.Sequential(OrderedDict([('fc1', nn.Linear(model.fc.in_features, hidden_units)),
                                     ('relu1', nn.ReLU()),
                                     ('d1', nn.Dropout(.2)),
                                     ('fc2', nn.Linear(hidden_units, hidden_units)),
                                     ('relu2', nn.ReLU()),
                                     ('d2', nn.Dropout(.2)),
                                     ('fc3', nn.Linear(hidden_units, hidden_units)),
                                     ('relu3', nn.ReLU()),
                                     ('d3', nn.Dropout(.2)),
                                     ('fc4', nn.Linear(hidden_units, n_classes)),
                                     ('output', nn.LogSoftmax(dim=1))
                                    ]))

    
    # Replacing the old classifier in the pretrained model
    model.fc = clf
    

In [18]:
# Training the model, leaning heavilty on Pytorch's tutorial for transfer learning
def train_model(model, optimizer, criterion, data_dict, num_epochs=2, calc_validation=True, device='cuda'):
    since = time.time()
    phases = ['train','valid'] if calc_validation else ['train']
    model.to(device)
        
    print('*' * 25)
    print('Starting training process')
    print('*' * 25)

    for epoch in tqdm_notebook(range(num_epochs)):
        print('\nEpoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        
        for phase in phases:
            model.train() if phase=='train' else model.eval() 
            running_loss = 0
            running_corrects = 0 
            
            print(phase)
            
            for inputs, labels in tqdm_notebook(data['dataloaders'][phase], leave=False):
                
                inputs, labels = inputs.to(device), labels.to(device)    # Move training data to device
                optimizer.zero_grad()                                    # Setting autograd to zero so it doesn't accumulate

                # Forward pass; for performance only use autograd during training
                with torch.set_grad_enabled(phase == 'train'):          
                    outputs = model.forward(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Update the optimizer
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()   

                # Update running loss and num_correct for the current batch
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            # Calculate stats for the current epoch and report to user
            epoch_loss = running_loss / data['dataset_sizes'][phase]
            epoch_acc = running_corrects.double() / data['dataset_sizes'][phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
      

In [ ]:
# Initializing the model
model = load_pretrained_model('resnet152')
replace_classifier(model)

In [ ]:
# Initializing criterion and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.00001)

# Starting the training process
train_model(model, optimizer, criterion, data, num_epochs=3)
save_checkpoint(model, optimizer, criterion, data, 'resnet152', current_version)

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [22]:
#  Calculating accuracy on test set
def calc_test_accuracy(model, data_dict):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    correct = 0
    total = 0

    with torch.no_grad(): # Turning autograd tracking off 

        for images, labels in tqdm_notebook(data_dict['dataloaders']['test']):

            images, labels = images.to(device), labels.to(device)    # Load images and labels into GPU
            outputs = model(images)                                  # Forward prop
            _, predicted = torch.max(outputs.data, 1)                # Get the label with the maximum likelihood
            total += labels.size(0)                                  # Get the number of processed data points
            correct += (predicted == labels).sum().item()            # Get the number of correct classified data points

    print('Accuracy of the network on all test images: %d %%' % (100 * correct / total))


In [24]:
calc_test_accuracy(model, data)

HBox(children=(IntProgress(value=0, max=13), HTML(value='')))


Accuracy of the network on all test images: 87 %


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [27]:
def save_checkpoint(model, optimizer, criterion, data_dict, arch, current_version):

    checkpoint = {'version':version,
                  'base_model': arch,
                  'classifier': model.fc,
                  'class_to_idx': data_dict['class_names'],
                  'optimizer': optimizer,
                  'criterion': criterion,
                  'classifier_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict()}

    torch.save(checkpoint, 'checkpoint_{}_v{}.pth'.format(arch, version))

In [28]:
save_checkpoint(model, optimizer, criterion, data, 'resnet152', current_version)

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [11]:
def load_from_checkpoint(filepath, last_device='cuda:0'):
    checkpoint = torch.load(filepath, map_location=last_device)
    
    # Downloading the basemodel
    if checkpoint['base_model'] == 'resnet152':
        model = models.resnet152(pretrained=True)
        for p in model.parameters():
            p.requires_grad = False
            
    elif checkpoint['base_model'] == 'resnet18':
        model = models.resnet18(pretrained=True)
        for p in model.parameters():
            p.requires_grad = False
            
    else:
        print('Add base model to loading function')
    
    # Replacing the base model's classifier and updating state dict
    model.fc = checkpoint['classifier']
    model.load_state_dict(checkpoint['classifier_state_dict'])
    
    # Adding the class_to_idx dictionary to the model
    model.class_to_idx = checkpoint['class_to_idx']
    
    # Creating the optimizer and updateing state dict
    optimizer = checkpoint['optimizer']
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    # Creating the criterion used for training
    criterion = checkpoint['criterion']
    
    # Computing the current version of the checkpoint
    version = checkpoint['version']+1
    
    # Return tuple
    return model, criterion, optimizer, version


In [21]:
model, criterion, optimizer, current_version = load_from_checkpoint('checkpoint_resnet152_v2.pth')

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # Get dimensions for the image
    width = image.size[0]
    height = image.size[1]

    # Resizing the image while keeping aspect ratios intact
    proper_size = (256, int(256*height/width)) if width<height else (int(256*width/height), 256)
    im_resized = image.resize(proper_size)
        
    # Center cropping the image
    x_center = proper_size[0]/2
    y_center = proper_size[1]/2
    box = (x_center-112, y_center-112, x_center+112, y_center+112)
    im_cropped = im_resized.crop(box)

    # Normalize color channels
    im_numpy = np.array(im_cropped)/255
    
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    im_numpy_norm = (im_numpy - mean)/ std
    
    # Transpose to move color dimesnion first
    im_numpy_norm_t = im_numpy_norm.transpose((2, 0, 1))
    
    return im_numpy_norm_t


To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
# Testing image processing function
pil_im = Image.open('flowers/train/85/image_04812.jpg') # Wide example
# pil_im = Image.open('flowers/train/85/image_04776.jpg') # Tall example

print('Original image:')
plt.imshow(pil_im)
plt.show()

print('Processed imaged:')
p_im = process_image(pil_im)
imshow(p_im)
plt.show()

print('Checking dimensions:')
print(p_im.shape)

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    # require no autograd for performance
    with torch.no_grad():
        
        pil_im = Image.open(image_path)                   # Load the orginal image
        tensor_im = torch.Tensor(process_image(pil_im))   # Change to numpy array and turn into tensor
        tensor_im.unsqueeze_(0)                           # Add an additional dimension for batch size
        tensor_im = tensor_im.to(device)                  # Move image to GPU
        output = model(tensor_im)                         # Do forward prop and save the output
        nll, idx = output.topk(topk)                      # Get the k best precitions and loss
        nll, idx = nll[0].tolist(), idx[0].tolist()       # Cast to python lists
        
        # Invert index for lookup
        idx_to_class = {v: k for k, v in model.class_to_idx.items()}
        classes = [idx_to_class.get(item) for item in idx]
        
        # calculate probabilities
        proba = np.exp(nll)
        
        return proba, classes
  
    

In [ ]:
# Testing my function's output
probs, classes = predict('flowers/train/85/image_04812.jpg', model)
print(probs)
print(classes)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
def predict_single_image(image_path, model):  
    probs, classes = predict(image_path, model)
    class_names = [cat_to_name[i] for i in classes]
    df = pd.DataFrame(probs,class_names, columns=['probability'])

    pil_im = Image.open(image_path)
    process_im = process_image(pil_im)

    # Creating subplots
    fig, (ax2, ax1) = plt.subplots(1,2, figsize=(7,4))

    # Adding image to top plot
    imshow(process_im, ax=ax1)
    fig.suptitle(class_names[0], y=.95, fontsize=20)
    ax1.axis('off')

    # Adding probabilities to bottom plot
    ax2.set_aspect(1/6)
    df.plot.barh(ax=ax2, legend=False)
    plt.show()


In [ ]:
example_path = 'personal_example.jpg'
predict_single_image(example_path, model)